In [18]:
import pandas as pd
import numpy as np
import scipy as sc
import csv
from scipy.optimize import minimize

In [19]:
# data preparation
data = pd.read_csv('metdata.txt', delim_whitespace = True)
data['Rich'] = (data['V9']>9).astype(int)
data['NE'] = (data['V8']==1).astype(int)
data['NC'] = (data['V8']==2).astype(int)
data['S'] = (data['V8']==3).astype(int)
data.head()
names_used = ['V3', 'V5', 'NE', 'NC', 'S']
cnst = True
if cnst == True:
    names_used.append('cnst')
    data['cnst'] = 1    
n_cols = len(names_used)
N = data.shape[0]

In [20]:
def logistic_prob(x):
    y = np.exp(x)/(1+np.exp(x))
    return y
    
    
def logistic_dens(x):
    y = np.exp(x)/(1+np.exp(x))**2
    return y

def log_like(beta, dt = data, names = names_used,  Y = 'Rich' , test = True):
    
    assert len(beta) == len(names), 'Dimensions do not match!'

    
    s = np.dot(dt[names].values, beta) # should be a vector which has scalar product of 
                                            # betas and Xs for each observation
    
    ss = logistic_prob(s) # apply logistic distribution to it
    
    l = (dt[Y].values)*np.log(ss) + (1 - dt[Y].values)*np.log(1-ss) # compute log likelihood
    L = np.mean(l)
    
    if test ==True:
        print(s.shape)
        print('Likelihood for first observation')
        print(l[0])
        return
     
    return L


def log_like_for_opt(beta):
    
    s = np.dot(data[names_used].values, beta) # should be a vector which has scalar product of 
                                            # betas and Xs for each observation
    
    ss = logistic_prob(s) # apply logistic distribution to it
    
    l = (data['Rich'].values)*np.log(ss) + (1 - data['Rich'].values)*np.log(1-ss) # compute log likelihood
    L = np.mean(l)

    return -L

def log_gradient(beta):
    
    s = np.dot(data[names_used].values, beta)
    ss = logistic_prob(s)
    sss = (data['Rich'] - ss).values
    
    g = (sss.reshape((N,1)))*(data[names_used].values)

    G = np.mean(g, axis=0)
    
    return -G

def log_hess(beta):
        
    s = np.dot(data[names_used].values, beta)
    ss = logistic_dens(s)
    sss = ss.reshape((N,1,1))
    X= (data[names_used].values).reshape((N,n_cols,1))
    Xt= (data[names_used].values).reshape((N,1,n_cols))
    
    XX = X*Xt
    
    h = (sss)*(XX)

    H = np.mean(h, axis=0)
    
    return H


In [21]:
### Question 2
log_like([0,0,0,0,0,0], data, names_used , test = False) # should be equal to log(1/2)~ -0.69

-0.6931471805599453

In [22]:
### Question 3
b0 = np.array([0,0,0,0,0,0])
out = minimize(log_like_for_opt, b0,jac=log_gradient,  method='BFGS' , options={'disp': True})
se = np.sqrt(np.diagonal(out['hess_inv']))
print('Coefficients')
print(out['x'])
print('Standard Errors')
print(se)

Optimization terminated successfully.
         Current function value: 0.363819
         Iterations: 35
         Function evaluations: 40
         Gradient evaluations: 40
Coefficients
[ 0.53868685  0.03776949  1.14130292  0.74272575 -0.10569418 -9.39251907]
Standard Errors
[ 1.2346292   0.38821234  8.42948306  8.62888656  9.47520572 23.28843015]


In [23]:
### Question 4

b0 = np.array([0,0,0,0,0,0])
out = minimize(log_like_for_opt, b0,jac=log_gradient, hess= log_hess, method='Newton-CG', 
               options={'xtol': 1e-6, 'disp': True})

print('Coefficients')
print(out['x'])
print('Standard Errors')
print(np.sqrt(np.diag(np.linalg.inv(log_hess(out['x'])))))


Optimization terminated successfully.
         Current function value: 0.363819
         Iterations: 16
         Function evaluations: 22
         Gradient evaluations: 37
         Hessian evaluations: 16
Coefficients
[ 0.53868602  0.03776939  1.14131     0.74271554 -0.10569443 -9.39250397]
Standard Errors
[ 1.26656882  0.41280129  8.85547074  9.0927367   9.94581997 23.88024379]
